In [2]:
!pip install langchain
!pip install tiktoken
!pip install sentence-transformers
 !pip install chromadb
 !pip install openAI


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 18.0 MB/s eta 0:00:00


In [3]:

from langchain.document_loaders import WebBaseLoader

# Initialize the WebBaseLoader with the URL of the document to be loaded
loader = WebBaseLoader("https://my.clevelandclinic.org/health/diseases/10946-cavities")

# Load the document and store it in the 'data' variable
data = loader.load()

In [6]:
import tiktoken

# Set up token encoding for the GPT-3.5 Turbo model
tiktoken.encoding_for_model('gpt-3.5-turbo')

<Encoding 'cl100k_base'>

In [7]:
tokenizer = tiktoken.get_encoding('cl100k_base')

# Define a function to calculate the token length of a given text
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

tiktoken_len("Dentin decay: Dentin is the layer just beneath your tooth enamel.")

15

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter with specified parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = tiktoken_len
)

In [9]:
# Split the loaded document into smaller chunks
chunks = text_splitter.split_documents(data)

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

# Specify the model name and additional arguments
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device' : 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

# Initialize HuggingFace Embeddings
hf = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
embed = hf.embed_documents(texts=['h','e'])

# Print the length of one of the embeddings to check its dimensions
print(len(embed[1]))

384


In [15]:
from langchain.vectorstores import Chroma

# Initialize Chroma vector database with chunks and HuggingFace embeddings
vectordb = Chroma.from_documents(chunks, hf)

In [16]:
# Perform a similarity search on the vector database
vectordb.similarity_search('bleeding gums', k=3)

[Document(page_content='sodas, juice and milk). The bacteria convert these carbohydrates into acids.Bacteria, acid, food and saliva mix to form dental plaque. This sticky substance coats your teeth.Without proper brushing and flossing, acids in plaque dissolve tooth enamel, creating cavities, or holes, in the enamel surface.What are the risk factors for cavities?Certain factors increase your risk of cavities, including:Dry mouth (xerostomia). Certain conditions (like Sjögren’s', metadata={'description': 'A cavity is a hole, or area of decay, in your tooth. Cavities form when acids in your mouth erode (wear down) your tooth enamel — your tooth’s hard, outer layer.', 'language': 'en', 'source': 'https://my.clevelandclinic.org/health/diseases/10946-cavities', 'title': 'Cavities (Tooth Decay): Symptoms, Causes & Treatment'}),
 Document(page_content='a tooth abscess can spread to surrounding tissues and other areas of your body. In rare cases, infection can even spread to your brain or to y

In [22]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Initialize a language model with ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo', openai_api_key='your api key', temperature=0.6)


# Initialize a RetrievalQA chain with the language model and vector database retriever
qa_chain = RetrievalQA.from_chain_type(llm, retriever= vectordb.as_retriever())


In [23]:

#Change the query to what you want to ask the LLM
query = 'What is enamel decay'

In [24]:
qa_chain({'query' : query})

#we got error due to used full free serivces openai

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}